In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import linear_model, preprocessing
import os
import glob
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import nltk
from nltk import tokenize
from nltk.stem import WordNetLemmatizer
import re 
# nltk.download('stopwords')
# nltk.download()
from nltk.corpus import stopwords 

In [0]:
nltk.download('punkt')
nltk.download('all')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora

True

In [0]:
stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'a', 'an', 'the', 'and', 'of', 'for', 'with', 'about', 'between', 'into', 'through', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'under', 'here', 'there', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'so', 'than', 'too', 'very', 'just']

In [0]:
def load_train_val(train,val):
  df= pd.read_csv(train,delimiter='\t',encoding='utf-8')
  tr_st=np.array(df['sentence'].values)
  tr_label=np.array(df['label_combined'].values)
  df= pd.read_csv(val,delimiter='\t',encoding='utf-8')
  val_st=np.array(df['sentence'].values)
  val_label=np.array(df['label_combined'].values)
  return tr_st,tr_label, val_st,val_label


In [0]:
def load_test(test):
  df= pd.read_csv(test,delimiter='\t',encoding='utf-8')
  ts_st=np.array(df['sentence'].values)
  ts_label=np.array(df['label_combined'].values)
  return ts_st,ts_label

In [0]:
def preprocess_sentences(X):
  documents = []
  stemmer = WordNetLemmatizer()
  for sen in range(0, len(X)):
    document = re.sub(r'\W', ' ', str(X[sen]))
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    document = re.sub(r'^"', '', document)
    document = re.sub(r'"$', '', document)
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    document = re.sub(r'^b\s+', '', document)
    document = document.lower()
    #     document = document.split()
    #     document = [stemmer.lemmatize(word) for word in document]
    #     document = ' '.join(document)
    documents.append(document)
  print(len(documents))
  return documents

In [0]:
def sent_to_features(documents):
  feature_docs=[]
  for document in documents:
    tokens=nltk.word_tokenize(document)
    bigrams = list(nltk.bigrams(tokens))
    #pos_bigrams = list(nltk.bigrams([pos for (word,pos) in nltk.pos_tag(tokens)]))
    document=[word+' '+pos for (word,pos) in nltk.pos_tag(tokens)]#+' '+pos
    document.extend([w1+'_'+w2 for (w1,w2) in bigrams])
    #     document.extend([w1+'_'+w2 for (w1,w2) in pos_bigrams])
    #     document.extend([pos for (word,pos) in nltk.pos_tag(tokens)])
    #     print(document)
    #     document = document.split()
    #     document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    feature_docs.append(document)
  return feature_docs

In [0]:
# documents = []
# stemmer = WordNetLemmatizer()
# for sen in range(0, len(X)):
#   document = re.sub(r'\W', ' ', str(X[sen]))
#   document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
#   document = re.sub(r'^"', '', document)
#   document = re.sub(r'"$', '', document)
#   document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
#   document = re.sub(r'\s+', ' ', document, flags=re.I)
#   document = re.sub(r'^b\s+', '', document)
#   document = document.lower()
#   #     document = document.split()
#   #     document = [stemmer.lemmatize(word) for word in document]
#   #     document = ' '.join(document)
#   documents.append(document)
# print(len(documents))

986


In [0]:
# import numpy as np

with open("glove.6B.300d.txt", "r", encoding='utf-8', errors='ignore') as lines:
    w2v = {line.split()[0]: np.array(list(map(float, line.split()[1:]))) for line in lines}
print(w2v['of'])
# def readWordvectors(wordvectorfile):
#   wordvectors = {}
#   vectorsize = 0
#   if ".gz" in wordvectorfile:
#     f = gzip.open(wordvectorfile, 'r')
#   else:
#     f = open(wordvectorfile, 'r',encoding='latin-1')
#   count = 0
#   for line in f:
#     if count == 0:
#       count += 1
#       continue
#     parts = line.split()
#     word = parts[0]
#     parts.pop(0)
#     wordvectors[word] = parts
#     vectorsize = len(parts)
#   f.close()
#   return [wordvectors, vectorsize]
# w2v,vectorsize= readWordvectors('vecs.txt')

In [0]:
len(w2v)

400000

In [0]:
import os    
from chardet import detect

def get_encoding_type(file):
    with open(file, 'rb') as f:
        rawdata = f.read().decode('latin-1')
    return detect(rawdata)['encoding']

from_codec = get_encoding_type("vecs.txt")
print(from_codec)

In [0]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(list(w2v.values())[1])

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.sum([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

In [0]:
len(list(w2v.values())[1])

# list(w2v.keys())[:10]

300

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
datapath = 'gdrive/My Drive/data/'
trainfile=datapath+'train_combined.tsv'
valfile=datapath+'val_combined.tsv'
testfile=datapath+'test_combined.txt'

In [0]:
#loading train and validation data
# trainfile='train.txt'
# valfile='validation.txt'
tr_st,tr_label, val_st,val_label=load_train_val(trainfile,valfile)

#preprocessing
tr_st = preprocess_sentences(tr_st)
val_st = preprocess_sentences(val_st)

#feature extraction
tr_st = sent_to_features(tr_st)
val_st = sent_to_features(val_st)

print('Train size: ',len(tr_st))
print('Val size: ',len(val_st))

1776
592
Train size:  1776
Val size:  592


In [0]:
print(tr_st[1])

friends NNS and CC adversaries NNS abroad RB were VBD asking VBG whether IN america NN had VBD lost VBN its PRP$ nerve NN friends_and and_adversaries adversaries_abroad abroad_were were_asking asking_whether whether_america america_had had_lost lost_its its_nerve


In [0]:
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'a', 'an', 'the', 'and', 'of', 'for', 'with', 'about', 'between', 'into', 'through', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'under', 'here', 'there', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'so', 'than', 'too', 'very', 'just']


In [0]:
from sklearn.feature_extraction.text import CountVectorizer  
vectorizer = CountVectorizer( stop_words=stop_words)  #, stop_words=stopwords.words('english')
documents = np.concatenate((tr_st,val_st))
print(len(documents))
vectorizer.fit(documents)


2368


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', '...t', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'so', 'than', 'too', 'very', 'just'],
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [0]:
tr_st = vectorizer.transform(tr_st)
val_st = vectorizer.transform(val_st)

In [0]:
print(tr_st.shape)
# print(X[0][:1000])

(1776, 39395)


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer  
tfidfconverter = TfidfVectorizer()#min_df=2, max_df=0.8, stop_words=stopwords.words('english')
documents = np.concatenate((tr_st,val_st))
print(len(documents))
tfidfconverter.fit(documents) 
tr_st = tfidfconverter.transform(tr_st)
val_st = tfidfconverter.transform(val_st)

2368


In [0]:
print(tr_st.shape)
# print(Xt[0][:1000])


(1776, 39471)


In [0]:
meanevconverter= MeanEmbeddingVectorizer(w2v)
tr_st = meanevconverter.transform(tr_st)
print('tranformed tr data: ', tr_st.shape)
val_st = meanevconverter.transform(val_st)
print('tranformed val data', val_st.shape)

tranformed tr data:  (1776, 300)
tranformed val data (592, 300)


In [0]:
print(tr_st[20])

In [0]:
# from sklearn.model_selection import train_test_split  
# Xtr, Xts, ytr, yts = train_test_split(X, yf, test_size=0.3, random_state=42)

In [0]:
from sklearn import svm

svc = svm.SVC(probability=False, kernel="linear", C=2.8, gamma=.0073,verbose=10)

In [0]:
svc.fit(tr_st,tr_label)

[LibSVM]

SVC(C=2.8, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.0073, kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=10)

In [0]:
yhat_ts = svc.predict(val_st)

In [0]:
yhat_tr = svc.predict(tr_st)

In [0]:
acc = np.mean(yhat_tr == tr_label)
print('Training Accuaracy = {0:f}'.format(acc))

Training Accuaracy = 1.000000


In [0]:
acc = np.mean(yhat_ts == val_label)
print('Accuaracy = {0:f}'.format(acc))

Accuaracy = 0.657095


In [0]:
from sklearn.metrics import classification_report
target_names = ['class 0', 'class 1','class 2','class 3']
print(classification_report(val_label, yhat_ts, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.41      0.44      0.42        73
     class 1       0.70      0.62      0.66       183
     class 2       0.70      0.85      0.77       279
     class 3       0.44      0.12      0.19        57

   micro avg       0.66      0.66      0.66       592
   macro avg       0.56      0.51      0.51       592
weighted avg       0.64      0.66      0.64       592



In [0]:
print("Future:")
unique, counts = np.unique(np.concatenate((tr_ft,val_ft)), return_counts=True)
print("Total: ",dict(zip(unique, counts)))
unique, counts = np.unique(tr_ft, return_counts=True)
print("Train: ",dict(zip(unique, counts)))
unique, counts = np.unique(val_ft, return_counts=True)
print("Test: ",dict(zip(unique, counts)))


Future:
Total:  {0.0: 1012, 1.0: 1356}
Train:  {0.0: 756, 1.0: 1020}
Test:  {0.0: 256, 1.0: 336}


In [0]:
C_test = [0.1,1,2.5,2.8,5,10]
gam_test = [0.001,0.01,0.1]

nC = len(C_test)
ngam = len(gam_test)
acc = np.zeros((nC,ngam))

In [0]:
# Xtr, Xts, ytr, yts = train_test_split(X, yf, test_size=0.3, random_state=42)

# TODO:  Print the accuracy matrix
for index_C , C in enumerate(C_test):
  for index_G , gamma in enumerate(gam_test):
    svc2 = svm.SVC(probability=True, kernel="linear", C=C, gamma=gamma, verbose=10)
    svc2.fit(tr_st,tr_label)
    yhat_testing_cross = svc2.predict(val_st)
    acc[index_C , index_G ] = np.mean(yhat_testing_cross == val_label)
    print(C,gamma)
    print('Acc', acc[index_C,index_G])

[LibSVM]0.1 0.001
Acc 0.47128378378378377
[LibSVM]0.1 0.01
Acc 0.47128378378378377
[LibSVM]0.1 0.1
Acc 0.47128378378378377
[LibSVM]1 0.001
Acc 0.6570945945945946
[LibSVM]1 0.01
Acc 0.6570945945945946
[LibSVM]1 0.1
Acc 0.6570945945945946
[LibSVM]2.5 0.001
Acc 0.6570945945945946
[LibSVM]2.5 0.01
Acc 0.6570945945945946
[LibSVM]2.5 0.1
Acc 0.6570945945945946
[LibSVM]2.8 0.001
Acc 0.6570945945945946
[LibSVM]2.8 0.01
Acc 0.6570945945945946
[LibSVM]2.8 0.1
Acc 0.6570945945945946
[LibSVM]5 0.001
Acc 0.6570945945945946
[LibSVM]5 0.01
Acc 0.6570945945945946
[LibSVM]5 0.1
Acc 0.6570945945945946
[LibSVM]10 0.001
Acc 0.6570945945945946
[LibSVM]10 0.01
Acc 0.6570945945945946
[LibSVM]10 0.1
Acc 0.6570945945945946


In [0]:
optimal_index_C , optimal_index_Gamma = np.unravel_index(acc.argmax(),acc.shape)

print("Maximum Accuracy",acc[optimal_index_C][optimal_index_Gamma])
opt_c_f = C_test[optimal_index_C]
opt_g_f = gam_test[optimal_index_Gamma]
print("Optimal C:", C_test[optimal_index_C])
print('Optimal Gamma:' , gam_test[optimal_index_Gamma])

Maximum Accuracy 0.8023648648648649
Optimal C: 0.1
Optimal Gamma: 0.001


In [0]:
testfile='test.txt'
ts_st,ts_ft,ts_pt=load_test(testfile)

#preprocessing
ts_st = preprocess_sentences(ts_st)

#feature extraction
ts_st = sent_to_features(ts_st)



593


In [0]:

ts_st = vectorizer.transform(ts_st)

In [0]:
ts_st = tfidfconverter.transform(ts_st)


In [0]:
tr_val_st = vectorizer.transform(documents)

In [0]:
tr_val_st = tfidfconverter.transform(documents)

In [0]:
documents = np.concatenate((tr_st,val_st))
print(len(documents))
tr_val_st = meanevconverter.transform(documents)
ts_st = meanevconverter.transform(ts_st)

2368


In [0]:

svc2 = svm.SVC(probability=True, kernel="linear", C=0.1, gamma=0.001, verbose=10)
print(tr_val_st.shape,tr_ft.shape,val_ft.shape)
svc2.fit(tr_val_st,np.concatenate((tr_ft,val_ft)))


(2368, 39395) (1776,) (592,)
[LibSVM]

SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=10)

In [0]:
yhat_testing = svc2.predict(ts_st)
accu = np.mean(yhat_testing == ts_ft)
print('Test Accuracy: ',accu)

Test Accuracy:  0.806070826306914


In [0]:
target_names = ['class 0', 'class 1']
print(classification_report(ts_ft, yhat_testing, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.00      0.00      0.00       293
     class 1       0.51      1.00      0.67       300

   micro avg       0.51      0.51      0.51       593
   macro avg       0.25      0.50      0.34       593
weighted avg       0.26      0.51      0.34       593



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
